## Models  
https://huggingface.co/Organika/sdxl-detector  
https://huggingface.co/NYUAD-ComNets/NYUAD_AI-generated_images_detector  
https://huggingface.co/RishiDarkDevil/ai-image-det-resnet152

## Imports

In [1]:
from transformers import AutoModelForImageClassification, AutoProcessor
import torch
from torchvision import datasets, transforms
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

c:\Users\metet\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load Models

In [2]:
model1 = AutoModelForImageClassification.from_pretrained("Organika/sdxl-detector")
model2 = AutoModelForImageClassification.from_pretrained("NYUAD-ComNets/NYUAD_AI-generated_images_detector")
model3 = AutoModelForImageClassification.from_pretrained("RishiDarkDevil/ai-image-det-resnet152")

processor1 = AutoProcessor.from_pretrained("Organika/sdxl-detector")
processor2 = AutoProcessor.from_pretrained("NYUAD-ComNets/NYUAD_AI-generated_images_detector")
# processor3 = AutoProcessor.from_pretrained("RishiDarkDevil/ai-image-det-resnet152") # Processor does not exist, will transform manually

Some weights of ResNetForImageClassification were not initialized from the model checkpoint at RishiDarkDevil/ai-image-det-resnet152 and are newly initialized: ['classifier.1.bias', 'classifier.1.weight', 'embedder.embedder.convolution.weight', 'embedder.embedder.normalization.bias', 'embedder.embedder.normalization.num_batches_tracked', 'embedder.embedder.normalization.running_mean', 'embedder.embedder.normalization.running_var', 'embedder.embedder.normalization.weight', 'encoder.stages.0.layers.0.layer.0.convolution.weight', 'encoder.stages.0.layers.0.layer.0.normalization.bias', 'encoder.stages.0.layers.0.layer.0.normalization.num_batches_tracked', 'encoder.stages.0.layers.0.layer.0.normalization.running_mean', 'encoder.stages.0.layers.0.layer.0.normalization.running_var', 'encoder.stages.0.layers.0.layer.0.normalization.weight', 'encoder.stages.0.layers.0.layer.1.convolution.weight', 'encoder.stages.0.layers.0.layer.1.normalization.bias', 'encoder.stages.0.layers.0.layer.1.normaliz

In [31]:
model3.eval()

ResNetForImageClassification(
  (resnet): ResNetModel(
    (embedder): ResNetEmbeddings(
      (embedder): ResNetConvLayer(
        (convolution): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (normalization): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (activation): ReLU()
      )
      (pooler): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (encoder): ResNetEncoder(
      (stages): ModuleList(
        (0): ResNetStage(
          (layers): Sequential(
            (0): ResNetBottleNeckLayer(
              (shortcut): ResNetShortCut(
                (convolution): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
                (normalization): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              )
              (layer): Sequential(
                (0): ResNetConvLayer(
                  (convolution): Conv2d(64

## Load Datasets

In [4]:
dir = "archive/datasets"

def load_dataset(dir, batch_size):
    dataset = datasets.ImageFolder(dir)
    loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=False)
    return loader

artTest = load_dataset(dir + "/art_512x512/test", 32)
facesTest = load_dataset(dir + "/faces_512x512/test", 32)

# transforming (only for model3)
transform = transforms.Compose([
    transforms.Resize((224, 224)),   
    transforms.ToTensor(),            
    transforms.Normalize(                
        mean=[0.485, 0.456, 0.406],         
        std=[0.229, 0.224, 0.225]
    ),
])

## Test Models